Method of undetermined coefficients for CIA baseline model

Метод неопределённых коэффициентов для модели CIA

Задаём скаляры!

In [8]:
theta = 0.36
beta = 0.99
delta = 0.025
A = 1.72
B = -2.5805
gamma = 0.95
pi = 0.48
# gbar = 1.02
gbar = 1
rbar = 1 / beta - 1 + delta
wbar = (1 - theta) * (rbar / theta)^(theta / (theta - 1))
cbar = -beta * wbar / gbar / B
kbar = cbar / (rbar / theta - delta)
hbar = (rbar / theta)^(1 / (1 - theta)) * kbar
pbar = 1 / cbar
k = 2

Задаём кучу матриц :)

In [26]:
A = [kbar 0 0 0]'

In [40]:
B = [-(rbar + 1 - delta) * kbar (1 - theta)  -theta 0]'

In [11]:
C = [-rbar * kbar   -wbar * hbar  -wbar * hbar  -1 / pbar; 
       1 0 (theta - 1) 0;
       0 1 theta 0; 
       0  -1 0  -1] 

In [15]:
D = [0 0; 
    -1 0; 
    -1 0; 
    0 pi] 
F = [0]
G = [0] 
H = [0]

In [68]:
J = [beta * rbar -1 0 0]
K = [0 1 0 0]
L = [0 0]
M = [0 0]

1×2 Array{Int64,2}:
 0  0

In [69]:
N = [gamma 0;
    0 pi]

2×2 Array{Float64,2}:
 0.95  0.0 
 0.0   0.48

Ух! Считаем всякие вкусности:

In [43]:
a = F - J * inv(C) * A
b = -(J * inv(C) * B - G + K * inv(C) * A)
c = -K * inv(C) * B + H
discr = b^2 - 4 * a * c
p1 = (-b + sqrt(discr)) / (2 * a)
p2 = (-b - sqrt(discr)) / (2 * a)

1×1 Array{Float64,2}:
 0.941817

Из двух корней выберем тот, что лежит внутри единичного круга, если такой есть. 

Здесь код **julia** немного отличается от **matlab**. В **matlab** "abs(p1)", в **julia** "abs(p1[1, 1])". **Julia** — девушка более строгого поведения/типизирования и запрещает сравнивать массивы со скаляром при проверке условия. А **matlab** вольно трактует массив размера один на один как скаляр и разрешает сравнивать скаляр со скаляром.

In [66]:
if (abs(p1[1, 1]) < 1)
    P = p1
    elseif (abs(p2[1, 1]) < 1)
    P = p2
else
    println("I'm sorry, both roots are unstable.")
end 

1×1 Array{Float64,2}:
 0.941817

Продолжаем разговор :)

Снова небольшая разница с **matlab**: присваивание в два скаляра сразу реализуется без квадратных скобок. **Julia**, как подобает воспитанной девушке, понимает разницу между двумя скалярами и матрицей размера 1 на 2.

In [76]:
R = -inv(C) * (A * P + B)
q1 = kron(N', (F - J * inv(C) * A)) + 
         kron(eye(k), (J * R + F * P + G - K * inv(C) * A))
q2_old = (J * inv(C) * D - L) * N + K * inv(C) * D - M
r, c = size(q2_old)
q2 = reshape(q2_old, c, r) 
Q = inv(q1) * q2
Q = Q'
S = -inv(C) * (A * Q + D)

4×2 Array{Float64,2}:
  1.94173   -0.0555083
  0.470274   0.0312234
  1.47146   -0.0867318
 -0.470274   0.448777 